Only able to pull the stop data for select stops, as with testing earlier. Will try to figure out what to do about the route info.

In [1]:
import os
import requests

import pandas as pd

In [11]:
# API Key and Endpoint
endpoint = "https://bustime.mta.info/api/where/stops-for-location.json"
params = {
    "key": os.getenv("MTA_API_KEY"),
    "lat": 40.715,  # Central Eastern Queens
    "lon": -73.825,
    "radius": 1000
}

# Fetch Data
response = requests.get(endpoint, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")

Status Code: 200


In [12]:
print(response.text)

{"code":200,"currentTime":1736175353939,"data":{"limitExceeded":false,"stops":[{"code":"502158","direction":"NW","id":"MTA_502158","lat":40.719693,"locationType":0,"lon":-73.818023,"name":"MAIN ST/78 RD","routes":[{"agency":{"disclaimer":"","email":"","fareUrl":"","id":"MTA NYCT","lang":"en","name":"MTA New York City Transit","phone":"718-330-1234","privateService":false,"timezone":"America/New_York","url":"http://www.mta.info"},"color":"006CB7","description":"via 20 Av / Main St","id":"MTA NYCT_Q20A","longName":"College Point - Jamaica","shortName":"Q20A","textColor":"FFFFFF","type":3,"url":""},{"agency":{"disclaimer":"","email":"","fareUrl":"","id":"MTA NYCT","lang":"en","name":"MTA New York City Transit","phone":"718-330-1234","privateService":false,"timezone":"America/New_York","url":"http://www.mta.info"},"color":"006CB7","description":"via 14 Av / Main St","id":"MTA NYCT_Q20B","longName":"College Point - Jamaica","shortName":"Q20B","textColor":"FFFFFF","type":3,"url":""}],"wheelc

In [14]:
# Structure Data
stops = [
    {
        "Stop ID": stop['id'],
        "Name": stop['name'],
        "Latitude": stop['lat'],
        "Longitude": stop['lon']
    }
    for stop in data['data']['stops']
]

# Save to CSV
stops_df = pd.DataFrame(stops)
stops_df.to_csv("../data/eastern_queens_stops.csv", index=False)

In [ ]:
# Attempt to collect route and stop info together. WIll need to work through some erros.

# List of routes to collect
route_ids = [
    "MTA NYCT_Q1", "MTA NYCT_Q43", "MTA NYCT_Q76", "MTA NYCT_Q36",
    "MTA NYCT_Q110", "MTA NYCT_Q46", "MTA NYCT_Q20A", "MTA NYCT_Q20B",
    "MTA NYCT_Q44", "MTA NYCT_Q30", "MTA NYCT_Q31", "MTA NYCT_Q27",
    "MTA NYCT_Q88", "MTA NYCT_Q4", "MTA NYCT_Q5", "MTA NYCT_Q84",
    "MTA NYCT_Q85", "MTA NYCT_Q83", "MTA NYCT_Q112", "MTA NYCT_Q12",
    "MTA NYCT_Q13", "MTA NYCT_Q28", "MTA NYCT_Q2", "MTA NYCT_Q3", "MTA NYCT_Q77"
]

# Function to fetch route and stop data
def fetch_route_and_stops(route_id):
    endpoint = "https://bustime.mta.info/api/where/route-details.json"
    params = {"key": api_key, "id": route_id}
    response = requests.get(endpoint, params=params)
    data = response.json()
    
    # Extract route information
    route_info = {
        "Route ID": data['data']['entry']['id'],
        "Short Name": data['data']['entry']['shortName'],
        "Description": data['data']['entry']['description']
    }
    
    # Extract stop information
    stops = [
        {
            "Route ID": route_info["Route ID"],
            "Stop ID": stop['id'],
            "Stop Name": stop['name'],
            "Latitude": stop['lat'],
            "Longitude": stop['lon'],
            "Sequence": i
        }
        for i, stop in enumerate(data['data']['references']['stops'])
    ]
    
    return route_info, stops

# Collect and save data
all_routes = []
all_stops = []
for route_id in route_ids:
    print(f"Fetching data for {route_id}...")
    route_info, stops = fetch_route_and_stops(route_id)
    all_routes.append(route_info)
    all_stops.extend(stops)

# Save to CSV
pd.DataFrame(all_routes).to_csv("../data/routes.csv", index=False)
pd.DataFrame(all_stops).to_csv("../data/stops.csv", index=False)
print("Data saved to 'routes.csv' and 'stops.csv'")